##Load Google Drive

In [13]:
#connect google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#access current folder
!ls
import os
os.chdir('/content/drive/My Drive/Project/情感分析')
!ls
#!git clone https://github.com/AimeeLee77/keyword_extraction

LDA_sensitive.ipynb  微博数据.xls   情感词汇本体.xlsx
stopWord.txt	     情感词库.xlsx
LDA_sensitive.ipynb  微博数据.xls   情感词汇本体.xlsx
stopWord.txt	     情感词库.xlsx


##sensitive corpus

In [15]:
import pandas as pd
import jieba
import time

def sensitive_corpus():
  '''
  生成positive和negative的词库
  '''
  df = pd.read_excel('情感词汇本体.xlsx')
  print(df.head(2))
  Happy = []
  Good = []
  Surprise = []
  Anger = []
  Sad = []
  Fear = []
  Disgust = []
  for idx, row in df.iterrows():
      if row['情感分类'] in ['PA', 'PE']:
          Happy.append(row['词语'])
      if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
          Good.append(row['词语']) 
      if row['情感分类'] in ['PC']:
          Surprise.append(row['词语'])     
      if row['情感分类'] in ['NA']:
          Anger.append(row['词语'])    
      if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
          Sad.append(row['词语'])
      if row['情感分类'] in ['NI', 'NC', 'NG']:
          Fear.append(row['词语'])
      if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
          Disgust.append(row['词语'])
  Positive = Happy + Good +Surprise
  Negative = Anger + Sad + Fear + Disgust
  print('情绪词语列表整理完成')   
  return Positive, Negative

def emotion_caculate(text):
  '''
  计算每一段文本的情感分数
  '''
  positive = 0
  negative = 0
  wordlist = jieba.lcut(text)
  wordset = set(wordlist)
  wordfreq = []
  for word in wordset:
      freq = wordlist.count(word)
      if word in Positive:
          positive+=freq
      if word in Negative:
          negative+=freq
  emotion_info = {
      'positive': positive,
      'negative': negative,
  }
  indexs = ['positive', 'negative']
  return pd.Series(emotion_info, index=indexs)

Positive, Negative = sensitive_corpus()

   词语 词性种类  词义数  词义序号 情感分类  强度  极性 辅助情感分类  强度.1  极性.1 Unnamed: 10  Unnamed: 11
0  脏乱  adj  1.0   1.0   NN   7   2    NaN   NaN   NaN         NaN          NaN
1  糟报  adj  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN          NaN
情绪词语列表整理完成


##sensitive analysis

In [16]:
!pip3 install pandarallel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandarallel import pandarallel

def data_pp():
  '''
  为每条文本赋予对应的情绪分数
  '''
  weibo = pd.read_excel('微博数据.xls',sheet_name = 1).astype(str).iloc[:,2:3]#sheet name 为不同的表
  weibo = weibo.dropna()
  print(weibo.head())
  #并行初始化
  pandarallel.initialize()
  start = time.time()   
  emotion_df = weibo['正文'].parallel_apply(emotion_caculate)
  #emotion_df = weibo_df['review'].apply(emotion_caculate)
  end = time.time()
  print(end-start)
  print(emotion_df.head())
  #将情绪分数转化为二值
  emotion_df['po>ne'] = [1 if emotion_df['positive'][i]>=emotion_df['negative'][i] else 0 for i in range(len(emotion_df))]
  emotion_df.head()
  #将文本与对应的情感合并
  output_df = pd.concat([weibo, emotion_df['po>ne']], axis=1)
  output_df.head()
  return output_df

def split_data(output_df):
  '''
  把两种情感分别保存在两个表中
  '''
  po = []
  ne = []
  for i, value in enumerate(output_df['po>ne']):
    if value==1:
      po.append(output_df['正文'][i])
    else:
      ne.append(output_df['正文'][i])
  print('len(po)\n',len(po))
  print('len(ne)\n',len(ne))
  po = pd.DataFrame(po)
  ne = pd.DataFrame(ne)
  print('po.head()\n',po.head())
  return po, ne
  
output_df = data_pp()
po, ne = split_data(output_df)

                                                  正文
0  通州早就比首都拥堵了，这下该挤爆了 //@RODGA:然后通州也变北京那样[doge][do...
1  北京减负，疏散非首都功能，最该迁出去的就是央企总部。像中外运长航、三峡集团、中国能源建设集团...
2  不是疏散北京非首都功能吗？ //【宝沃国产项目落户北京 斥资50亿搞研发】宝沃国产项目落户北...
3  【疏解北京】《纲要》明确了京津冀区域的目标定位，即要打造成以首都为核心的世界级城市群。这表明...
4  #生态京津冀#“以减法换加法”。近年来，本市非首都功能产业疏散的减法换来园林绿化建设的加法。...
INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
11.48845100402832
   positive  negative
0         1         1
1         2         1
2         0         1
3         4         2
4         5         0
len(po)
 233
len(ne)
 19
po.head()
                                                    0
0  通州早就比首都拥堵了，这下该挤爆了 //@RODGA:然后通州也变北京那样[doge][do...
1  北京减负，疏散非首都功能，最该迁出去的就是央企总部。像中外运长航、三峡集团、中国能源建设集团...
2  【疏解北京】《纲要》明确了京津冀区域的目标定位，即要打造成以首都为核心的世界级城市群。这表明...
3  #生态京津冀#“以减法换加法”。近年来，本市非首都功能产业疏散的减法换来园林绿化建设的加法。...
4  北京市为了疏散人口，转移非首都功能，立志要把全部或部分行政事业单位迁出，劳民伤财啊？能不能不...


##data pre-processing

In [17]:
import sys,codecs
import jieba.posseg
import jieba.analyse

def dataPrepos(text, stopkey):
  '''
  数据预处理操作：分词，去停用词，词性筛选
  '''
  l = []
  pos = ['n', 'nz', 'v', 'vd', 'vn', 'l', 'a', 'd']  # 定义选取的词性
  seg = jieba.posseg.cut(text)  # 分词
  for i in seg:
      if i.word not in stopkey and i.flag in pos:  # 去停用词 + 词性筛选
          l.append(i.word)
  return l

def data_pp(data, stopkey):
  '''
  对每一行文本预处理
  '''
  idList, abstractList = data.index, data[0]
  corpus = [] # 将所有文档输出到一个list中，一行就是一个文档
  for index in range(len(idList)):
      text = abstractList[index] # 
      text = dataPrepos(text,stopkey) # 文本预处理
      #text = ",".join(text) # 连接成字符串，空格分隔
      corpus.append(text)
  print(corpus[1:3])
  return corpus

stopkey = [w.strip() for w in codecs.open('stopWord.txt', 'r').readlines()]
data = po#选择positive的情感
corpus = data_pp(data, stopkey)

[['减负', '疏散', '首都', '功能', '最该', '迁出去', '总部', '集团', '能源建设', '集团', '集团', '石油', '铝业', '必要', '总部', '设在', '迁来', '领导', '经济', '利益', '欢迎', '总部', '加剧', '企业', '总部', '功能', '首都', '功能'], ['疏解', '纲要', '京津冀', '区域', '目标', '定位', '造成', '首都', '核心', '世界级', '城市群', '表明', '低端', '产业', '疏散', '发展', '区域', '辐射', '作用', '高端', '产业', '重点', '疏散', '首都', '功能', '一定', '时间段', '主城区', '人口', '降下来', '重点', '在于', '人口', '重新聚集']]


##LDA 

In [18]:
from gensim import corpora,models
import re

def lda_result(corpus):
  # 主题分析
  neg_dict = corpora.Dictionary(corpus)
  neg_corpus = [neg_dict.doc2bow(i) for i in corpus]
  neg_lda = models.LdaModel(neg_corpus,num_topics = 3,id2word = neg_dict)
  neg_theme = neg_lda.show_topics()
  # print(neg_theme)#展示主题
  pattern = re.compile(r'[\u4e00-\u9fa5]+')
  print(pattern.findall(neg_theme[0][1]))# 取出高频词
  pos_key_words =[]
  for i in range(3):
    pos_key_words.append(pattern.findall(neg_theme[i][1]))
  pos_key_words = pd.DataFrame(data=pos_key_words,index=['主题1','主题2','主题3'])
  print(pos_key_words)  
  return pos_key_words

record_topic = lda_result(corpus)

['投资', '首都', '发展', '功能', '疏散', '产业', '经济', '物流', '国际', '京津冀']
      0   1   2   3   4   5   6   7   8    9
主题1  投资  首都  发展  功能  疏散  产业  经济  物流  国际  京津冀
主题2  功能  首都  疏散  投资  雄安  发展  产业  企业  经济   物流
主题3  投资  发展  产业  功能  经济   雾  首都  疏散  企业   大哥
